### Here we clean the weather csv and merge it with the existing energy generation csv

In [16]:
"source: https://open-meteo.com/en/docs/historical-weather-api?start_date=2023-01-01&end_date=2026-01-01&timezone=Europe%2FBerlin&location_mode=csv_coordinates&csv_coordinates=50.110924,8.682127+++%23+Frankfurt+am+Main%0A52.52437,13.41053++++%23+Berlin%0A48.13743,11.57549++++%23+Munich%0A53.551086,9.993682+++%23+Hamburg%0A50.93333,6.95++++++++%23+Cologne+%28Köln%29&hourly=temperature_2m,relative_humidity_2m,cloud_cover,wind_speed_100m,wind_direction_100m,precipitation,soil_moisture_0_to_7cm,shortwave_radiation#api_response"

'source: https://open-meteo.com/en/docs/historical-weather-api?start_date=2023-01-01&end_date=2026-01-01&timezone=Europe%2FBerlin&location_mode=csv_coordinates&csv_coordinates=50.110924,8.682127+++%23+Frankfurt+am+Main%0A52.52437,13.41053++++%23+Berlin%0A48.13743,11.57549++++%23+Munich%0A53.551086,9.993682+++%23+Hamburg%0A50.93333,6.95++++++++%23+Cologne+%28Köln%29&hourly=temperature_2m,relative_humidity_2m,cloud_cover,wind_speed_100m,wind_direction_100m,precipitation,soil_moisture_0_to_7cm,shortwave_radiation#api_response'

In [2]:
# 50.110924,8.682127   # Frankfurt am Main
# 52.52437,13.41053    # Berlin
# 48.13743,11.57549    # Munich
# 53.551086,9.993682   # Hamburg
# 50.93333,6.95        # Cologne (Köln)

In [1]:
import pandas as pd

path = "new_data/open-meteo-forecast.csv"

# Find the time-series header line
ts_header_row = None
with open(path, "r", encoding="utf-8") as f:
    for i, line in enumerate(f):
        if line.strip().lower().startswith("location_id,time"):
            ts_header_row = i
            break

if ts_header_row is None:
    raise ValueError("Could not find time-series header line starting with 'location_id,time'")

# Read ONLY the time-series block
df_weather_long = pd.read_csv(path, skiprows=ts_header_row)

# Parse and index
df_weather_long["time"] = pd.to_datetime(df_weather_long["time"])
df_weather_long = df_weather_long.sort_values(["location_id", "time"]).reset_index(drop=True)

df_weather_long.head()

,location_id,time,temperature_2m (°C),relative_humidity_2m (%),cloud_cover (%),wind_speed_120m (km/h),wind_direction_120m (°),precipitation (mm),soil_moisture_1_to_3cm (m³/m³),shortwave_radiation (W/m²)
0,0,2023-01-01 00:00:00,12.8,78,100,36.4,210,0.0,0.302,0.0
1,0,2023-01-01 01:00:00,12.5,76,63,35.2,214,0.0,0.302,0.0
2,0,2023-01-01 02:00:00,11.6,82,21,36.0,211,0.0,0.302,0.0
3,0,2023-01-01 03:00:00,11.0,87,0,36.3,209,0.0,0.302,0.0
4,0,2023-01-01 04:00:00,11.0,85,0,34.7,213,0.0,0.285,0.0


In [2]:
weather_vars = [c for c in df_weather_long.columns if c not in ("location_id", "time")]

df_mean = (
    df_weather_long
    .groupby("time")[weather_vars]
    .mean()
    .add_prefix("wx_mean__")
)

df_std = (
    df_weather_long
    .groupby("time")[weather_vars]
    .std(ddof=0)
    .add_prefix("wx_std__")
)

df_weather_agg = df_mean.join(df_std).sort_index()
df_weather_agg.head()

,wx_mean__temperature_2m (°C),wx_mean__relative_humidity_2m (%),wx_mean__cloud_cover (%),wx_mean__wind_speed_120m (km/h),wx_mean__wind_direction_120m (°),wx_mean__precipitation (mm),wx_mean__soil_moisture_1_to_3cm (m³/m³),wx_mean__shortwave_radiation (W/m²),wx_std__temperature_2m (°C),wx_std__relative_humidity_2m (%),wx_std__cloud_cover (%),wx_std__wind_speed_120m (km/h),wx_std__wind_direction_120m (°),wx_std__precipitation (mm),wx_std__soil_moisture_1_to_3cm (m³/m³),wx_std__shortwave_radiation (W/m²)
time,,,,,,,,,,,,,,,,
2023-01-01 00:00:00,13.10,67.6,94.0,37.68,212.6,0.00,0.3222,0.0,2.959730,10.781466,10.526158,14.639044,11.993331,0.00,0.041082,0.0
2023-01-01 01:00:00,13.66,61.4,88.8,40.56,212.8,0.00,0.3212,0.0,2.391318,10.150862,13.614698,10.782690,13.702555,0.00,0.040187,0.0
2023-01-01 02:00:00,13.24,63.6,70.2,39.74,210.4,0.02,0.3204,0.0,2.326027,11.943199,35.851918,13.080765,16.144349,0.04,0.039500,0.0
2023-01-01 03:00:00,12.66,66.8,60.0,38.30,221.0,0.12,0.3216,0.0,2.390481,13.614698,48.989795,12.519585,12.441865,0.24,0.041774,0.0
2023-01-01 04:00:00,12.82,66.4,63.2,37.16,232.4,0.08,0.3124,0.0,2.644542,11.876026,45.353721,13.160486,27.244082,0.16,0.043821,0.0


In [3]:
df_weather_agg.to_csv("new_data/weather_forecast.csv")

### To merge weather forecast data with weather historical data

In [4]:
# df_hist: your main dataframe with energy + past weather (wx_*)
# df_fcst: forecast weather dataframe indexed by time (valid time), same hourly index

df_hist = pd.read_csv("new_data/germany_energy_with_weather_CI.csv")
df_fcst = pd.read_csv("new_data/weather_forecast.csv")
df_fcst = df_fcst.copy()

# rename all forecast columns with a clear prefix
df_fcst = df_fcst.add_prefix("wxf__")   # e.g. wxf__wx_mean__temperature_2m (°C)

# or if you want cleaner names:
# df_fcst.columns = [c.replace("wx_", "wxf_") for c in df_fcst.columns]

df = df_hist.join(df_fcst, how="left")
df.to_csv("new_data/germany_energy_with_weather_with_forecast_CI.csv")

In [6]:
df.columns

Index(['time', 'Biomass', 'Energy storage', 'Fossil Brown coal/Lignite',
       'Fossil Coal-derived gas', 'Fossil Gas', 'Fossil Hard coal',
       'Fossil Oil', 'Fossil Oil shale', 'Fossil Peat', 'Geothermal',
       'Hydro Pumped Storage', 'Hydro Run-of-river and pondage',
       'Hydro Water Reservoir', 'Marine', 'Nuclear', 'Other',
       'Other renewable', 'Solar', 'Waste', 'Wind Offshore', 'Wind Onshore',
       'total_power_mw', 'emissions_weighted', 'carbon_intensity',
       'wx_mean__temperature_2m (°C)', 'wx_mean__relative_humidity_2m (%)',
       'wx_mean__cloud_cover (%)', 'wx_mean__wind_speed_100m (km/h)',
       'wx_mean__wind_direction_100m (°)', 'wx_mean__precipitation (mm)',
       'wx_mean__soil_moisture_0_to_7cm (m³/m³)',
       'wx_mean__shortwave_radiation (W/m²)', 'wx_std__temperature_2m (°C)',
       'wx_std__relative_humidity_2m (%)', 'wx_std__cloud_cover (%)',
       'wx_std__wind_speed_100m (km/h)', 'wx_std__wind_direction_100m (°)',
       'wx_std__precipita

### Merge with energy generation dataframe

In [7]:
import pandas as pd

ENERGY_PATH = "new_data/germany_2325_ci.csv"   # change to your file

df_energy = pd.read_csv(
    ENERGY_PATH,
    parse_dates=["Time"],          # change if your column is named differently
)

df_energy = df_energy.set_index("Time").sort_index()
df_energy.index.name = "time"

In [8]:
def to_local_naive(idx: pd.DatetimeIndex, tz: str = "Europe/Berlin") -> pd.DatetimeIndex:
    """
    Convert DatetimeIndex to naive local time (drop tz info).
    - If idx is tz-aware: convert to tz then drop tz.
    - If idx is tz-naive: assume it's already local wall time.
    """
    if idx.tz is not None:
        return idx.tz_convert(tz).tz_localize(None)
    return idx

df_energy.index = to_local_naive(df_energy.index)
df_weather_agg.index = to_local_naive(df_weather_agg.index)

In [10]:
df_energy.index = df_energy.index.floor("h")
df_weather_agg.index = df_weather_agg.index.floor("h")

In [11]:
# Remove duplicates if any
df_energy = df_energy[~df_energy.index.duplicated(keep="first")]
df_weather_agg = df_weather_agg[~df_weather_agg.index.duplicated(keep="first")]

# Quick sanity prints
print("Energy range:", df_energy.index.min(), "→", df_energy.index.max(), "rows:", len(df_energy))
print("Weather range:", df_weather_agg.index.min(), "→", df_weather_agg.index.max(), "rows:", len(df_weather_agg))

Energy range: 2023-01-01 00:00:00 → 2025-12-31 23:00:00 rows: 26304
Weather range: 2023-01-01 00:00:00 → 2026-01-01 23:00:00 rows: 26328


In [12]:
df_merged = df_energy.join(df_weather_agg, how="left")

In [13]:
wx_cols = [c for c in df_merged.columns if c.startswith("wx_")]
print("Weather NaN fraction:", df_merged[wx_cols].isna().mean().mean())

Weather NaN fraction: 0.0


In [14]:
OUT_PATH = "new_data/germany_energy_with_weather.csv"
df_merged.to_csv(OUT_PATH, index=True)
print("Saved:", OUT_PATH)

Saved: new_data/energy_with_weather.csv


In [17]:
df_merged.columns

Index(['Biomass', 'Energy storage', 'Fossil Brown coal/Lignite',
       'Fossil Coal-derived gas', 'Fossil Gas', 'Fossil Hard coal',
       'Fossil Oil', 'Fossil Oil shale', 'Fossil Peat', 'Geothermal',
       'Hydro Pumped Storage', 'Hydro Run-of-river and pondage',
       'Hydro Water Reservoir', 'Marine', 'Nuclear', 'Other',
       'Other renewable', 'Solar', 'Waste', 'Wind Offshore', 'Wind Onshore',
       'total_power_mw', 'emissions_weighted', 'carbon_intensity',
       'wx_mean__temperature_2m (°C)', 'wx_mean__relative_humidity_2m (%)',
       'wx_mean__cloud_cover (%)', 'wx_mean__wind_speed_100m (km/h)',
       'wx_mean__wind_direction_100m (°)', 'wx_mean__precipitation (mm)',
       'wx_mean__soil_moisture_0_to_7cm (m³/m³)',
       'wx_mean__shortwave_radiation (W/m²)', 'wx_std__temperature_2m (°C)',
       'wx_std__relative_humidity_2m (%)', 'wx_std__cloud_cover (%)',
       'wx_std__wind_speed_100m (km/h)', 'wx_std__wind_direction_100m (°)',
       'wx_std__precipitation (mm